In [ ]:
#Installation of all the required lib here
!pip install oandapyV20
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for oandapyV20: filename=oandapyV20-0.7.2-py3-none-any.whl size=69779 sha256=f087556028b0da90586a71c431a954d21755fef0b67278e123d45ad96537d082
  Stored in directory: /root/.cache/pip/wheels/8e/69/ab/a6da84a85b9bf3b5a98ca15c6c52b7854f32b10c70fe0531a1
Successfully built oandapyV20
--2024-05-21 01:45:40--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2024-05-21 01:45:40--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sou

In [ ]:
import json
from oandapyV20 import API    # the client
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.orders as orders
import pandas as pd
import talib
import plotly.graph_objects as go


In [ ]:
accountID = "101-011-24509333-001"
access_token = "2092774d87027059265e897f7452eaa9-d4a0390dbef600c8119e62ab83b0b5a2"

# Create an API context
api = API(access_token=access_token)

# Define parameters for the historical data request
params = {
    "count": 5000,  # number of candles
    "granularity": "H1",  # 1-hour candles
    "price": "M"  # Midpoint prices
}

# Create and send the request
r = instruments.InstrumentsCandles(instrument="GBP_USD", params=params)
response = api.request(r)

prices = [(candle['time'], candle['mid']['o'],candle['mid']['h'],candle['mid']['l'],candle['mid']['c']) for candle in response['candles']]
df = pd.DataFrame(prices, columns=['Time','Open','High','Low', 'Close'])

# Convert the time to a pandas datetime format
df['Time'] = pd.to_datetime(df['Time'])

df=pd.merge(df,pd.DataFrame(response),left_index=True, right_index=True,how = 'inner')
if 'candles' in df.columns:
    df = df.drop(columns='candles')

df=df[['instrument', 'Time', 'granularity', 'Open', 'High', 'Low', 'Close']]
df['Open'] = df['Open'].astype(float)
df['High'] = df['High'].astype(float)
df['Low'] = df['Low'].astype(float)
df['Close'] = df['Close'].astype(float)
data =df

In [ ]:
#USING SMA CROSS OVER
df = df.copy()
df= df.iloc[100:200]
for number in range(8,100,8):
  df[f'MA_{number}'] = talib.SMA(df['Close'].values, timeperiod=number)


In [ ]:
fig = go.Figure()
fig.add_trace(go.Candlestick(x=df['Time'],
                             open=df['Open'],
                             high=df['High'],
                             low=df['Low'],
                             close=df['Close'],
                             name='Candlestick',
                            line=dict(width=1), opacity=1,
                            increasing_fillcolor='#24A06B',
                            decreasing_fillcolor="#CC2E3C",
                            increasing_line_color='#2EC886',
                            decreasing_line_color='#FF3A4C'))
fig.add_trace(go.Scatter(x=df['Time'], y=df['MA_8'],
                         mode='lines',
                         line=dict(color='blue', width=2),
                         name='MA8'))

fig.add_trace(go.Scatter(x=df['Time'], y=df['MA_16'],
                         mode='lines',
                         line=dict(color='yellow', width=2),
                         name='MA16'))
fig.update_layout(width=1500,height=500,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True)

fig.update_layout(title='Candlestick Chart with MA5',
                  xaxis_title='Time',
              yaxis_title='Price',)


fig.show()


In [ ]:
#USING SSL here
def SSL(df, ma_type='SMA', lb_period=10):
    if ma_type == 'SMA':
        sslu = df['High'].rolling(window=lb_period).mean()
        ssld = df['Low'].rolling(window=lb_period).mean()
    elif ma_type == 'EMA':
        sslu = df['High'].ewm(span=lb_period, adjust=False).mean()
        ssld = df['Low'].ewm(span=lb_period, adjust=False).mean()
    elif ma_type == 'LWMA':
        weights = range(1, lb_period + 1)
        sslu = df['High'].rolling(window=lb_period).apply(lambda x: (x * weights).sum() / weights.sum(), raw=True)
        ssld = df['Low'].rolling(window=lb_period).apply(lambda x: (x * weights).sum() / weights.sum(), raw=True)
    elif ma_type == 'SSMA':
        sslu = (df['High'].rolling(window=lb_period).sum() / lb_period).rolling(window=lb_period).sum() / lb_period
        ssld = (df['Low'].rolling(window=lb_period).sum() / lb_period).rolling(window=lb_period).sum() / lb_period
    else:
        raise ValueError("Invalid moving average type. Please choose from 'SMA', 'EMA', 'LWMA', or 'SSMA'.")

    df['smaHigh'] = sslu
    df['smaLow'] = ssld

    return df

SSL_df = data.copy()
SSL_df = SSL(SSL_df, ma_type='SMA', lb_period=20)

In [ ]:
SSL_df

,instrument,Time,granularity,Open,High,Low,Close,smaHigh,smaLow
0,GBP_USD,2023-07-31 16:00:00+00:00,H1,1.28550,1.28616,1.28524,1.28582,NaN,NaN
1,GBP_USD,2023-07-31 17:00:00+00:00,H1,1.28582,1.28673,1.28504,1.28516,NaN,NaN
2,GBP_USD,2023-07-31 18:00:00+00:00,H1,1.28516,1.28534,1.28362,1.28376,NaN,NaN
3,GBP_USD,2023-07-31 19:00:00+00:00,H1,1.28376,1.28388,1.28318,1.28341,NaN,NaN
4,GBP_USD,2023-07-31 20:00:00+00:00,H1,1.28342,1.28371,1.28316,1.28360,NaN,NaN
...,...,...,...,...,...,...,...,...,...
4995,GBP_USD,2024-05-20 21:00:00+00:00,H1,1.27064,1.27078,1.27029,1.27078,1.270824,1.269977
4996,GBP_USD,2024-05-20 22:00:00+00:00,H1,1.27078,1.27094,1.27066,1.27088,1.270828,1.270027
4997,GBP_USD,2024-05-20 23:00:00+00:00,H1,1.27088,1.27111,1.27074,1.27098,1.270846,1.270062
4998,GBP_USD,2024-05-21 00:00:00+00:00,H1,1.27098,1.27118,1.27064,1.27092,1.270875,1.270091


In [ ]:
#SSL INDICATOR [[[WORKS FUCKIIGG SUPER FINE]]]


def calculate_ssl_down(row):

    if row['Close'] < row['smaLow']:
        return row['smaHigh']
    else:
        return row['smaLow']

def calculate_ssl_up(row):
    if row['Close'] < row['smaLow']:
        return row['smaLow']
    else:
        return row['smaHigh']

def calculate_ssl(df, period):

    df["ATR"] = talib.ATR(high = df['High'],low = df['Low'],close = df['Close'], timeperiod=14)
    df['smaHigh'] = df['High'].rolling(window=period).mean()
    df['smaLow'] = df['Low'].rolling(window=period).mean()
    df['sslDown'] = df.apply(calculate_ssl_down, axis=1)
    df['sslUp'] = df.apply(calculate_ssl_up, axis=1)
    df['baseline'] = df['Close'].rolling(window = 8).mean()
    df['Position'] = df.apply(lambda row: "G" if row['smaLow'] == row['sslDown'] else "SELL", axis = 1)
    df['SIGNAL'] = df['Position'] != df['Position'].shift(1)
    df['BUY_1'] = (df['SIGNAL'] == True) & (df['Close'] > df['baseline'])
    df['Sell_1'] = (df['SIGNAL'] == True) & (df['Close'] < df['baseline'])
    df['atrup'] = df['Close'] + df['ATR']
    df['atrdown'] = df['Close'] - df['ATR']

    return df


In [ ]:
a = calculate_ssl(SSL_df, 10)
sig = a.iloc[100:300]
sig




,instrument,Time,granularity,Open,High,Low,Close,smaHigh,smaLow,sslDown,...,Position,SIGNAL,check,check1,BUY_1,Sell_1,ATR,atrup,atrdown,baseline
100,GBP_USD,2023-08-04 20:00:00+00:00,H1,1.27440,1.27566,1.27410,1.27524,1.276762,1.274024,1.274024,...,G,False,False,False,False,False,0.002125,1.277365,1.273115,1.276369
101,GBP_USD,2023-08-06 21:00:00+00:00,H1,1.27530,1.27625,1.27472,1.27472,1.277352,1.274544,1.274544,...,G,False,False,False,False,False,0.002082,1.276802,1.272638,1.276177
102,GBP_USD,2023-08-06 22:00:00+00:00,H1,1.27466,1.27566,1.27466,1.27532,1.277368,1.275182,1.275182,...,G,False,False,False,False,False,0.002005,1.277325,1.273315,1.276139
103,GBP_USD,2023-08-06 23:00:00+00:00,H1,1.27528,1.27528,1.27455,1.27490,1.277061,1.275209,1.277061,...,SELL,True,False,True,False,True,0.001917,1.276817,1.272983,1.275836
104,GBP_USD,2023-08-07 00:00:00+00:00,H1,1.27490,1.27614,1.27442,1.27587,1.276891,1.275146,1.275146,...,G,True,False,False,True,False,0.001903,1.277773,1.273967,1.275549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,GBP_USD,2023-08-16 23:00:00+00:00,H1,1.27296,1.27324,1.27258,1.27276,1.274201,1.272702,1.272702,...,G,False,False,False,False,False,0.001630,1.274390,1.271130,1.272963
296,GBP_USD,2023-08-17 00:00:00+00:00,H1,1.27274,1.27277,1.27163,1.27207,1.273867,1.272488,1.273867,...,SELL,True,False,True,False,True,0.001595,1.273665,1.270475,1.272809
297,GBP_USD,2023-08-17 01:00:00+00:00,H1,1.27208,1.27224,1.27112,1.27200,1.273519,1.272163,1.273519,...,SELL,False,False,False,False,False,0.001561,1.273561,1.270439,1.272569
298,GBP_USD,2023-08-17 02:00:00+00:00,H1,1.27200,1.27230,1.27140,1.27162,1.273243,1.271995,1.273243,...,SELL,False,False,False,False,False,0.001514,1.273134,1.270106,1.272469


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig_ssl = make_subplots(rows=2, cols=1, shared_xaxes=True)

# Candlestick trace
fig_ssl.add_trace(go.Candlestick(
    x=sig['Time'],
    open=sig['Open'],
    high=sig['High'],
    low=sig['Low'],
    close=sig['Close'],
    name='Candlestick',
    line=dict(width=1),
    opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',
    decreasing_line_color='#FF3A4C'  # Plot on the primary y-axis
), row=1, col=1)

# ATR trace
fig_ssl.add_trace(go.Scatter(
    x=sig['Time'],
    y=sig['atrup'],
    name='ATRU',
    mode='lines',
    line=dict(color='RED', width=1)  # Plot on the secondary y-axis
), row=1, col=1)

fig_ssl.add_trace(go.Scatter(
    x=sig['Time'],
    y=sig['atrdown'],
    name='ATRD',
    mode='lines',
    line=dict(color='YELLOW', width=1)  # Plot on the secondary y-axis
), row=1, col=1)

fig_ssl.add_trace(go.Scatter(
    x=sig['Time'],
    y=sig['baseline'],
    name='BASELINE',
    mode='lines',
    line=dict(color='yellow', width=1)  # Plot on the secondary y-axis
), row=1, col=1)

fig_ssl.add_trace(go.Scatter(
    x=sig['Time'],
    y=sig['sslUp'],
    name='SSLU',
    line=dict(color='blue', width=2),  # Plot on the secondary y-axis
), row=2, col=1)

# SSLD trace
fig_ssl.add_trace(go.Scatter(
    x=sig['Time'],
    y=sig['sslDown'],
    name='SSLD',
    line=dict(color='green', width=2),  # Plot on the secondary y-axis
), row=2, col=1)

# Markers for 'check1'
fig_ssl.add_trace(go.Scatter(
    x=sig[sig['check1'] == True]['Time'],
    y=sig[sig['check1'] == True]['High'],
    mode='markers',
    name='Check1',
    marker=dict(color='yellow', size=8, symbol='triangle-down'),
), row=1, col=1)

# Markers for 'check'
fig_ssl.add_trace(go.Scatter(
    x=sig[sig['check'] == True]['Time'],
    y=sig[sig['check'] == True]['Low'],
    mode='markers',
    name='Check',
    marker=dict(color='blue', size=8, symbol='triangle-up'),
), row=1, col=1)

fig_ssl.update_layout(
    width=1500,
    height=500,
    margin=dict(l=10, r=10, b=10, t=10),
    font=dict(size=10, color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e",
    xaxis_title='Time',
    yaxis_title='Price',
    title='Candlestick Chart with MA5'
)

fig_ssl.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,
    fixedrange=True,
    rangeslider=dict(visible=True)
)

fig_ssl.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)

fig_ssl.show()
